In [194]:
from shapely.geometry import Point
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

loading may data - pickle and csv


In [210]:
%%time
may_loc = pd.read_pickle("../data/may.pkl")
# may_trip = pd.read_csv("../data/may_trip.csv")

Wall time: 2.12 s


## SECTION: DICTIONARY OF TIME-SEGMENT DATAFRAMES
This section makes a dictionary of dataframes. The dataframes can be "chunked up" into different time segments. The main things you need to adjust: the start and end variables, and the delta variable, which designates how "big" a "time chunk" should be. Datetime docs here: https://docs.python.org/3/library/datetime.html

In [212]:
# may_trip['pubTimeStamp'] = pd.to_datetime(may_trip['pubTimeStamp'])
# may_trip['pubTimeStamp']
# may_trip.head()
# may_trip.info()
may_loc.head()

# Austin PEay's company dict, built from pickle file
#company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}


,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname
0,2019-05-01 00:01:41.247,36.136822,-86.799877,PoweredLIRL1,93.0,0
1,2019-05-01 00:01:41.247,36.191252,-86.772945,PoweredXWRWC,35.0,0
2,2019-05-01 00:01:41.247,36.144752,-86.806293,PoweredMEJEH,90.0,0
3,2019-05-01 00:01:41.247,36.162056,-86.774688,Powered1A7TC,88.0,0
4,2019-05-01 00:01:41.247,36.150973,-86.783109,Powered2TYEF,98.0,0


In [211]:
#make a list of datetimes with datetime function 
start = datetime(2019, 5, 10, 0, 0)
end = datetime(2019, 5, 11, 0, 0)
delta = timedelta(days=1)
timelist = []
timenamelist = []
while start <= end:
    timelist.append(start)
    timenamelist.append(start.strftime("%Y-%m-%d-%H-%M"))
    start += delta
# print(timelist)
# print(timenamelist)

In [196]:
print(timenamelist)

['2019-05-10-00-00', '2019-05-11-00-00']


In [213]:
#goal of function: using list of time frames, in this case, days, make a dataframe containing
#info about the location for each scooter, for each day


#make dictionary of dataframes using base dataframe and a user-defined segments_of_time_list
#from cell above
def make_day_df(datetime_list, timenames_list, dataframe, colname):
    resultslist = {} #turn into dictionary
    x = 0
    y = 1
    z = 0
    while x in range(0,len(datetime_list)-1):
        starttime = datetime_list[x]
        endtime = datetime_list[y]
        mask = (dataframe[colname] > starttime) & (dataframe[colname] <= endtime)
        framename = dataframe.loc[mask]
        x += 1
        y += 1
        print(x)
        resultslist[timenames_list[z]] = framename
        z += 1
    return resultslist
        
the_output = make_day_df(timelist, timenamelist, may_loc, "pubdatetime")


1


In [214]:
the_output.keys()

dict_keys(['2019-05-10-00-00'])

In [216]:
output_df = the_output['2019-05-10-00-00']
the_output['2019-05-10-00-00']['sumdid'].nunique()

3044

#### To Do
must figure out a way to narrow down the output_df. Basically, if the scooters check in every 5 minutes, I don't want to count the same scooter every time it checks in from the same location. Right now, the only filtering that has been done is by a set time period - still need to process that time period to get a more accurate picture of how many unique scooters are in a specific location during a spectific timeframe

##### Goal: to have a dataframe which does not contain multiple entries for the same sumdid if it remains at the same lat and long.
Basically, I'm only interested in mapping the number of unique scooters available within a given timeframe

In [222]:
output_df['sumdid'].value_counts() == 3

PoweredV7AZAO3WJGWCX    False
PoweredTB6JVKPTKJCQL    False
PoweredUCLK2G4J226JB    False
PoweredCFXWO2CWJYEAX    False
Powered4HT53YMZPD2JF    False
                        ...  
PoweredG6TP6            False
PoweredDCP9Y            False
Powered523737           False
Powered5R5UK            False
Powered267317           False
Name: sumdid, Length: 3044, dtype: bool

## NEW SECTION - GEOSPATIAL BELOW

This section below is experimental geospatial stuff, adapted from our geospatial notebook for the most part. IF you want to mess with this bit, make sure you load the notebook in the geospatial enivronment.

In [92]:
promise_zones = gpd.read_file('../data/MDHA_Promise_Zones/Export_Output_5.shp')
print(promise_zones.crs)
promise_zones= promise_zones.to_crs('EPSG:4326')
print(promise_zones)

epsg:2274
   OBJECTID  ZONE_ID    SHAPE_STAr     SHAPE_STLe  \
0         1        1  1.053728e+08   49364.601403   
1         2        2  7.670585e+07   45226.018917   
2         3        3  2.985488e+08   75207.067487   
3         4        4  2.713638e+08   74348.596054   
4         5        5  3.928178e+08  106661.712102   
5         6        6  9.535996e+07   59256.923574   

                                            geometry  
0  POLYGON ((-86.76297 36.19364, -86.76297 36.193...  
1  POLYGON ((-86.77838 36.14889, -86.77823 36.148...  
2  POLYGON ((-86.70771 36.16723, -86.70758 36.166...  
3  POLYGON ((-86.74735 36.13036, -86.74728 36.130...  
4  POLYGON ((-86.78490 36.23358, -86.78452 36.232...  
5  POLYGON ((-86.85223 36.16417, -86.85204 36.164...  


In [47]:
# zipcodes = gpd.read_file('../../da5-geospatial-python-workshop-derektprice/data/zipcodes.geojson')
# print(zipcodes.crs)
# zipcodes.head()

epsg:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [93]:
#adding geomtery column to a single output_df with lambda and point
output_df['geometry'] = output_df.apply(lambda x: Point(x['longitude'], 
                                                         x['latitude']), axis=1)
output_df.head(3)

C:\Users\Derek\AppData\Local\Temp/ipykernel_14176/3068456515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['geometry'] = output_df.apply(lambda x: Point(x['longitude'],


,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,POINT (-86.77045 36.12148)
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,POINT (-86.770228 36.121393)
154605,2019-05-01 07:00:03.897,36.144292,-86.811540,Powered341,90.0,2,POINT (-86.81153999999999 36.144292)


In [94]:
#make geo dataframe out of single output dataframe
output_df_as_geodf = gpd.GeoDataFrame(output_df,
                           crs = promise_zones.crs,
                           geometry = output_df['geometry'])
output_df_as_geodf.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,POINT (-86.77045 36.12148)
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,POINT (-86.77023 36.12139)
154605,2019-05-01 07:00:03.897,36.144292,-86.811540,Powered341,90.0,2,POINT (-86.81154 36.14429)
154606,2019-05-01 07:00:03.897,36.121616,-86.770332,Powered351,61.0,2,POINT (-86.77033 36.12162)
154607,2019-05-01 07:00:03.897,36.121575,-86.770093,Powered759,41.0,2,POINT (-86.77009 36.12157)


In [95]:
#spatial join output geo df with promise zone geodf or zipcode geodf
# output_geodf_and_promise = gpd.sjoin(output_df_as_geodf, promise_zones, op="within")
output_geodf_and_promise.head()
# output_geodf_and_zip = gpd.sjoin(output_df_as_geodf, zipcodes, op="within")
# output_geodf_and_zip.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry,index_right,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,POINT (-86.77045 36.12148),3,4,4,2.713638e+08,74348.596054
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,POINT (-86.77023 36.12139),3,4,4,2.713638e+08,74348.596054
154606,2019-05-01 07:00:03.897,36.121616,-86.770332,Powered351,61.0,2,POINT (-86.77033 36.12162),3,4,4,2.713638e+08,74348.596054
154607,2019-05-01 07:00:03.897,36.121575,-86.770093,Powered759,41.0,2,POINT (-86.77009 36.12157),3,4,4,2.713638e+08,74348.596054
154608,2019-05-01 07:00:03.897,36.121421,-86.770544,Powered658,98.0,2,POINT (-86.77054 36.12142),3,4,4,2.713638e+08,74348.596054


In [52]:
#number of rows in promise zone id or in each zip
# output_geodf_and_promise['ZONE_ID'].value_counts()
output_geodf_and_zip['zip'].value_counts()

37203    15278
37212     5535
37204     4107
37206     3084
37219     2956
37201     2328
37208     2064
37210     1636
37213     1485
37240      464
37209      414
37217      312
37218      240
37232      239
37228      216
37215      191
37216      144
37207      120
37205      120
37211      120
37013      120
37115       48
37214       48
37220       24
37138       24
Name: zip, dtype: int64

In [ ]:
#plotting output onto promise zones
leg_kwds = {'title': 'Promise Zone', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 1}

ax = promise_zones.plot(figsize = (8, 10), cmap='Set1', edgecolor = 'black',
              legend = True, legend_kwds = leg_kwds)
output_geodf_and_promise.plot(ax = ax, column='sumdid')
plt.show();

In [ ]:
#plotting output onto zip codes
ax = zipcodes.plot(figsize = (8, 10), cmap="Set2")
output_geodf_and_zip.plot(ax = ax, column='sumdid')
plt.show();

trying it with folium

In [97]:
# need centroid first - centroid of promise zones. I'm just picking a random one. Be careful - need to reverse long and lat
promise_center = promise_zones.geometry.centroid[1]
print(promise_center)

#flip it!!

zone_center = [promise_center.y, promise_center.x]
print(zone_center)

POINT (-86.77976341691249 36.138596506826)
[36.138596506826, -86.77976341691249]


C:\Users\Derek\AppData\Local\Temp/ipykernel_14176/2327727275.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  promise_center = promise_zones.geometry.centroid[1]


#### `itterows()` is a generator that iterates through the rows of a DataFrame and returns a tuple with the row id and row values. 
- Below, we are printing the row values for the first 3 row as we iterate through the GeoDataFrame of bus stops in 37207. 
- This idea will be helpful for creating our markers!

In [98]:
i = 0
for row_index, row_values in output_geodf_and_promise.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

index is 154603
values are:
 
pubdatetime    2019-05-01 07:00:03.897000
latitude                         36.12148
longitude                       -86.77045
sumdid                         Powered447
chargelevel                          66.0
companyname                             2
geometry       POINT (-86.77045 36.12148)
index_right                             3
OBJECTID                                4
ZONE_ID                                 4
SHAPE_STAr                  271363786.127
SHAPE_STLe                   74348.596054
Name: 154603, dtype: object
------------------------- 
index is 154604
values are:
 
pubdatetime      2019-05-01 07:00:03.897000
latitude                          36.121393
longitude                        -86.770228
sumdid                           Powered695
chargelevel                            96.0
companyname                               2
geometry       POINT (-86.770228 36.121393)
index_right                               3
OBJECTID                     

#### Create map_promise_zones again with additional features:
- add the promise zone area outline (promise_zones)
- iterate through the output_df_as_geodf to
    - create location from each lat and lng
    - create a popup from the stop column - not sure about this, need to adapt
    - create a custom icon if you want with [font-awesome](https://fontawesome.com/v4.7.0/icons/)
    - build a marker from each location and popup and (optionally) your custom icon 
    - add the marker to  `map_promise_zones`
- display `map_promise_zones`

In [99]:
output_geodf_and_promise.head()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,geometry,index_right,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe
154603,2019-05-01 07:00:03.897,36.121480,-86.770450,Powered447,66.0,2,POINT (-86.77045 36.12148),3,4,4,2.713638e+08,74348.596054
154604,2019-05-01 07:00:03.897,36.121393,-86.770228,Powered695,96.0,2,POINT (-86.77023 36.12139),3,4,4,2.713638e+08,74348.596054
154606,2019-05-01 07:00:03.897,36.121616,-86.770332,Powered351,61.0,2,POINT (-86.77033 36.12162),3,4,4,2.713638e+08,74348.596054
154607,2019-05-01 07:00:03.897,36.121575,-86.770093,Powered759,41.0,2,POINT (-86.77009 36.12157),3,4,4,2.713638e+08,74348.596054
154608,2019-05-01 07:00:03.897,36.121421,-86.770544,Powered658,98.0,2,POINT (-86.77054 36.12142),3,4,4,2.713638e+08,74348.596054


In [ ]:
#draw our zip code area: 37207
map_promise_zones = folium.Map(location = zone_center, zoom_start=12)

folium.GeoJson(promise_zones).add_to(map_promise_zones)

#iterate through stops_in_37207 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in output_geodf_and_promise.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['sumdid'])
    icon=folium.Icon(color="blue",icon="arrow-down", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_promise_zones)
map_promise_zones.save('../maps/promise_zone_map.html')

#display our map
map_promise_zones

#### If you have a lot of markers, you might want to use marker clusters
- Folium has `MarkerCluster()` and `FastMarkerCluster()`
- Construct a `MarkerCluster()` 
    - construct a `MarkerCluster()` and `.add_to()` the map
    - when you loop through the data and create markers add those to the marker cluster
- Construct a `FastMarkerCluster()` by passing a list of locations

In [ ]:
#draw our zip code area: 37207
cluster_map_promise_zones = folium.Map(location = zone_center, zoom_start=12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_map_promise_zones)


folium.GeoJson(promise_zones).add_to(cluster_map_promise_zones)

#iterate through stops_in_37207 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in output_geodf_and_promise.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['sumdid'])
    icon=folium.Icon(color="blue",icon="arrow-down", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(marker_cluster)

cluster_map_promise_zones.save('../maps/cluster_promise_zone_map.html')

#display our map
cluster_map_promise_zones